In [104]:
import pandas as pd
import glob
import re
from nltk import stem
from gensim import corpora, models, similarities


pd.set_option('display.max_columns', 500)
def concat(in_path):
    allFiles = glob.glob(in_path + "/*.txt")
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df_ = pd.read_table(file_,index_col=False, header=0)
        list_.append(df_)
    df = pd.concat(list_)
    df.to_csv(out_path+"articles.csv")
    print len(df)
    return df  

def extract_docs(df):
    docs_csv=out_path+"docs.csv"
    df=df[~df.AB.isnull()]
    df.AB.to_csv(docs_csv, index=False)
    print len(df)
    with open(docs_csv) as f:
        docs = f.readlines()    
    return docs

def pre(docs):
    porter = stem.porter.PorterStemmer()
    docs_prepared=out_path+'docs_prepared.csv'
    stop= open ('stopList.txt').read()
    stop_list = set(stop.replace("\n", " ").split())
    docs_cleaned = [[re.sub('[.!,;?()\W\d]+', '', word) for word in doc.lower().split()] for doc in docs]
    docs_stemed = [[porter.stem(word) for word in document if word not in stoplist] for document in docs_cleaned]
    with open(docs_prepared, 'w') as f:
        for doc in docs_stemed:f.write("%s\n" % ' '.join(doc))
    return docs_stemed


def ls():
    index = similarities.MatrixSimilarity(corpus_lsi) 
    with open(cleanedfile, 'r') as f:
        lines = f.read().splitlines()
        sims = open('08 LSI - similarities.csv', 'w')
        countdegree= open('09 LSI - degreecount.csv', 'w')
        sims.write('source,target,weight,type\n')
        for i in range(0,2427):
            doc=lines[i]    
            vec_bow = dictionary.doc2bow(doc.split())
            vec_tran = lsi[tfidf[vec_bow]]
            simsinx = index[vec_tran]
            count=0;
            #for j in range(0,len(simsinx)):
                #if simsinx[j]>.5: 
                    #count=count+1
            #countdegree.write(str(i+1)+','+str(count-1));countdegree.write('\n')
            for j in range(i+1,len(simsinx)):
                if simsinx[j]>.49:                                        
                    sims.write(str(i+1)+','+str(j+1)+','+str(simsinx[j])+',undirected');sims.write('\n')


In [75]:
in_path =r'../../data/raw/' # use your path
out_path= "../../data/clean/"
df=concat(in_path)

1354


In [79]:
docs=extract_docs(df)

1220


In [90]:
docs=pre(docs)

In [97]:
## documents_clean = [[word for word in document.split() ] for document in docs]
dictionary = corpora.Dictionary(docs)
dictionary.filter_extremes(no_below=5)
dictionary.compactify() 
print dictionary

class MyCorpus(object):
    def __iter__(self):
        for doc in docs:
            yield dictionary.doc2bow(doc)
corpus= MyCorpus() 
corpora.MmCorpus.serialize('deerwester.mm', corpus)
corpus = corpora.MmCorpus('deerwester.mm')
print corpus

Dictionary(2021 unique tokens: [u'', u'assimil', u'payoff', u'accur', u'deviat']...)
MmCorpus(1220 documents, 2021 features, 72036 non-zero entries)


In [98]:
tfidf = models.tfidfmodel.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100) 
corpus_lsi=lsi[corpus_tfidf]


In [108]:
    index = similarities.MatrixSimilarity(corpus_lsi) 
    lines = docs
    sims = open('08 LSI - similarities.csv', 'w')
    countdegree= open('09 LSI - degreecount.csv', 'w')
    sims.write('source,target,weight,type\n')
    for i in range(len(docs)):
        doc=lines[i]    
        vec_bow = dictionary.doc2bow(doc)
        vec_tran = lsi[tfidf[vec_bow]]
        simsinx = index[vec_tran]
        #count=0;
        #for j in range(0,len(simsinx)):
            #if simsinx[j]>.5: 
                #count=count+1
        #countdegree.write(str(i+1)+','+str(count-1));countdegree.write('\n')
        for j in range(i+1,len(simsinx)):
            if simsinx[j]>.49:                                        
                sims.write(str(i+1)+','+str(j+1)+','+str(simsinx[j])+',undirected');sims.write('\n')

In [126]:
df['keyword']=df.DE.apply(lambda x: "" if pd.isnull(x) else x.split(';')[0].lower())
keywords=df.keyword.groupby(df.keyword).count()
keywords.sort_values(ascending=False, inplace=True)
keywords.to_csv(out_path+'keywords.csv')
len(keywords)